In [1]:
import torch
import numpy as np

In [160]:
w = torch.tensor(2.8, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

x = torch.tensor([1.0, 2.0])

In [161]:
y_pred = w * x + b
print(f"y_pred: {y_pred}")

y_pred: tensor([3.8000, 6.6000], grad_fn=<AddBackward0>)


In [162]:
y_true = torch.tensor([10.0, 10.0])
loss = ((y_true - y_pred) ** 2).mean()
print(f"loss: {loss.item()}")

loss: 25.0


In [163]:
print(f"w before backward: w.grad = {w.grad}")

w before backward: w.grad = None


In [164]:
loss.backward()


In [165]:
print(f"apres backward: w.grad = {w.grad}")
print(f"apres backward: b.grad = {b.grad}")

apres backward: w.grad = -13.0
apres backward: b.grad = -9.600000381469727


In [166]:
learning_rate = 0.01
with torch.no_grad():
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad


In [167]:
print(f"nouveau poids: w = {w}, b = {b}")

nouveau poids: w = 2.929999828338623, b = 1.0959999561309814


In [168]:
import torch.nn as nn
perception = nn.Linear(1, 1)

print(f"poids: {perception.weight}")
print(f"biais: {perception.bias}")

poids: Parameter containing:
tensor([[-0.5488]], requires_grad=True)
biais: Parameter containing:
tensor([-0.5188], requires_grad=True)


In [169]:
# Créer un tensor de la forme (batch_size, in_features)
# perception attend des entrées de forme (N, 1) car nn.Linear(1,1)
x = torch.tensor([[3.0]])  # batch de 2 échantillons, 1 feature chacun
print(f"input: {x}")
y_prod = perception(x)
print(y_prod)

input: tensor([[3.]])
tensor([[-2.1652]], grad_fn=<AddmmBackward0>)


In [170]:
y_true = torch.tensor([[10.0]])
criterium = nn.MSELoss()
loss = criterium(y_prod, y_true)
print(f"loss: {loss.item()}")

loss: 147.99249267578125


In [171]:
loss.backward()

In [172]:
print(f"gradient du poids: {perception.weight.grad}")
print(f"gradient du biais: {perception.bias.grad}")

gradient du poids: tensor([[-72.9913]])
gradient du biais: tensor([-24.3304])


In [173]:
import torch.optim as optim
optimizer = optim.SGD(perception.parameters(), lr=0.01)


In [174]:
optimizer.step()


In [175]:
print(f"nouveau poids : {perception.weight} {perception.bias}")

nouveau poids : Parameter containing:
tensor([[0.1811]], requires_grad=True) Parameter containing:
tensor([-0.2755], requires_grad=True)


In [176]:
optimizer.zero_grad()

In [177]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [178]:
model = SimpleNet()

In [179]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [180]:
# Entrée : un batch d'un seul échantillon avec 2 features (SimpleNet attend 2 features)
x = torch.tensor([[1.0, 2.0]])  # forme (1, 2)
y = torch.tensor([[5.0]])  # forme (1, 1)
y_pred = model(x)
print(y_pred)

tensor([[0.4516]], grad_fn=<SigmoidBackward0>)


In [181]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose
import matplotlib.pyplot as plt


In [182]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [183]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [184]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [185]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits


In [186]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [187]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)

In [188]:
logits

tensor([[ 0.0642,  0.0778,  0.0075,  0.0554,  0.1373, -0.0359, -0.1160, -0.0350,
          0.1065, -0.0582]], grad_fn=<AddmmBackward0>)

In [189]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.1042, 0.1056, 0.0984, 0.1033, 0.1121, 0.0943, 0.0870, 0.0943, 0.1087,
         0.0922]], grad_fn=<SoftmaxBackward0>)

In [190]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([4])

In [191]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [198]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # Calcul des prédictions et de la perte
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [201]:
def test(dataloader, model, loss_fn):
    # Si on passe un dataset directement, on le transforme en DataLoader
    if isinstance(dataloader, torch.utils.data.Dataset):
        dataloader = DataLoader(dataloader, batch_size=64)

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = 100.0 * correct / size
    print(f"Test set: Accuracy: {accuracy:0.1f}%, Avg loss: {test_loss:>8f}")


In [202]:
epoche = 10
for t in range(epoche):
    print(f"epoche {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)
print("Done!")

epoche 1
-------------------------------
loss: 2.275935  [   64/60000]
loss: 2.256662  [ 6464/60000]
loss: 2.260169  [12864/60000]
loss: 2.267401  [19264/60000]
loss: 2.263662  [25664/60000]
loss: 2.229892  [32064/60000]
loss: 2.228990  [38464/60000]
loss: 2.224416  [44864/60000]
loss: 2.225823  [51264/60000]
loss: 2.218145  [57664/60000]
Test set: Accuracy: 53.5%, Avg loss: 2.216063
epoche 2
-------------------------------
loss: 2.231456  [   64/60000]
loss: 2.224496  [ 6464/60000]
loss: 2.208696  [12864/60000]
loss: 2.182045  [19264/60000]
loss: 2.187116  [25664/60000]
loss: 2.162013  [32064/60000]
loss: 2.183417  [38464/60000]
loss: 2.162772  [44864/60000]
loss: 2.133430  [51264/60000]
loss: 2.138137  [57664/60000]
Test set: Accuracy: 61.9%, Avg loss: 2.130822
epoche 3
-------------------------------
loss: 2.137528  [   64/60000]
loss: 2.092260  [ 6464/60000]
loss: 2.124086  [12864/60000]
loss: 2.079741  [19264/60000]
loss: 2.074819  [25664/60000]
loss: 2.073784  [32064/60000]
loss:

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ImageClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
torch_model = ImageClassifierModel()
# Create example inputs for exporting the model. The inputs should be a tuple of tensors.
example_inputs = (torch.randn(1, 1, 32, 32),)
onnx_program = torch.onnx.export(torch_model, example_inputs, dynamo=True)

[torch.onnx] Obtain model graph for `ImageClassifierModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ImageClassifierModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Obtain model graph for `ImageClassifierModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [4]:
onnx_program.save("image_classifier_model.onnx")

In [5]:
import onnx

onnx_model = onnx.load("image_classifier_model.onnx")
onnx.checker.check_model(onnx_model)